In [ ]:
import weave
weave.use_frontend_devmode()
from weave.legacy.weave import ops_arrow
from weave.legacy.weave.ecosystem import wandb

In [ ]:
import datetime
import random
random.seed(1)

min_x = datetime.datetime(2023, 3, 22, 11, 0, 0)
max_x = min_x + datetime.timedelta(hours=1)

users = ['nick', 'shawn', 'stacey', 'tim', 'danny']
model_versions = ['v1', 'v2']

num_requests = 10000

def random_datetime_in_range() -> datetime.datetime:
    min_timestamp = min_x.timestamp()
    max_timestamp = max_x.timestamp()
    random_timestamp = random.uniform(min_timestamp, max_timestamp)
    return datetime.datetime.fromtimestamp(random_timestamp)

def random_suggestion(n: int) -> str:
    result = ''
    for i in range(n):
        result += random.choice('abcdefghijklmnopqrstuvwxyz')
    return result


items = weave.save(
    ops_arrow.to_arrow([
        {
            "pred": random_suggestion(7),
            "time": random_datetime_in_range(),
            "user": random.choice(users),
            "version": random.choice(model_versions),
            "accepted": random.choice([1, 1, 1, 0, 0]),
        } 
        for _ in range(num_requests)
    ])
)

In [ ]:
panel = weave.legacy.weave.panels.Group(
    items={
        "num_preds": weave.legacy.weave.panels.LabeledItem(
            label="Predictions by version over time",
            item=wandb.TimeSeries(
                items,
                x=lambda item: item["time"],
                label=lambda item: item["version"],
                agg=lambda preds: preds.count(),
                min_x=min_x,
                max_x=max_x,
                mark="bar",
                axis_labels={"y": "num_preds"},
            ),
        ),
        "bottom_row": weave.legacy.weave.panels.Group(
            style="height: 400px;",
            preferHorizontal=True,
            items={
                "preds_by_user": weave.legacy.weave.panels.LabeledItem(
                    label="Predictions by user over time",
                    item=wandb.TimeSeries(
                        items,
                        x=lambda item: item["time"],
                        label=lambda item: item["user"],
                        agg=lambda preds: preds.count(),
                        min_x=min_x,
                        max_x=max_x,
                        mark="bar",
                        axis_labels={"y": "num_preds"},
                    ),
                ),
                "acceptance_rate": weave.legacy.weave.panels.LabeledItem(
                    label="Acceptance rate over time by version",
                    item=wandb.TimeSeries(
                        items,
                        x=lambda item: item["time"],
                        label=lambda item: item["version"],
                        agg=lambda preds: preds["accepted"].sum() / preds.count(),
                        min_x=min_x,
                        max_x=max_x,
                        mark="line",
                        axis_labels={"y": "acceptance_rate"},
                    ),
                ),
            },
        ),
    }
)


In [ ]:
panel